In [1]:
import pandas as pd
import numpy as np

RAW_PATH   = "../data/hospital_readmissions_dirty.csv"   # the “messed up” file
CLEAN_PATH = "../data/hospital_readmissions_clean.csv"   # will be created
VALID_REGIONS   = ["Northeast","Midwest","South","West"]
VALID_INSURANCE = ["Private","Medicare","Medicaid","Uninsured"]
VALID_LANGS     = ["English","Spanish","Chinese","Vietnamese","Arabic","Tagalog","Other"]

df = pd.read_csv(RAW_PATH)
df.head()


,Patient_ID,Age,Gender,Primary_Language,Diagnosis_Code,Admission_Type,Length_of_Stay,Readmitted,Cost_of_Stay,Hospital_Region,Insurance_Type,Comorbidities_Count,Admission_Date
0,1,58,Female,Spanish,A41,Urgent,10.1,0,21030.35,South,Medicare,3,2024-11-24
1,2,28,Female,Other,J44,Urgent,2.2,0,3227.00,Midwest,Medicaid,0,06/28/2024
2,3,0,Female,Spanish,F32,Emergency,9.5,0,8097.62,south,Medicare,0,02/04/2025
3,4,47,Male,Spanish,F32,Emergency,4.1,0,5488.64,Midwest,Private,1,07/05/2025
4,5,95,Other,Spanish,I10,Emergency,5.3,0,5384.99,West,Private,0,11/02/2024


In [2]:
print("Shape:", df.shape)
print("\nNulls per column:\n", df.isna().sum())
print("\nSample bad ages:", df.loc[(df["Age"]<0) | (df["Age"]>120), "Age"].value_counts().head())
print("LOS <= 0 or > 60:", ((df["Length_of_Stay"]<=0) | (df["Length_of_Stay"]>60)).sum())
print("Duplicates (exact):", df.duplicated().sum())


Shape: (10050, 13)

Nulls per column:
 Patient_ID               0
Age                      0
Gender                   0
Primary_Language       148
Diagnosis_Code         195
Admission_Type           0
Length_of_Stay           0
Readmitted               0
Cost_of_Stay           295
Hospital_Region          0
Insurance_Type         364
Comorbidities_Count      0
Admission_Date         105
dtype: int64

Sample bad ages: Age
-5      9
 150    1
Name: count, dtype: int64
LOS <= 0 or > 60: 50
Duplicates (exact): 50


In [3]:
def strip_lower(s): 
    return s.astype("string").str.strip().str.lower()

def strip_title(s): 
    return s.astype("string").str.strip().str.title()

# Hospital_Region: title-case (Northeast, Midwest, South, West)
df.loc[df["Hospital_Region"].notna(), "Hospital_Region"] = strip_title(df["Hospital_Region"].dropna())

# Insurance_Type: title-case (Private, Medicare, Medicaid, Uninsured)
df.loc[df["Insurance_Type"].notna(), "Insurance_Type"] = strip_title(df["Insurance_Type"].dropna())

# Primary_Language: title-case (English, Spanish, etc.)
df.loc[df["Primary_Language"].notna(), "Primary_Language"] = strip_title(df["Primary_Language"].dropna())


In [4]:
typo_fix = {
    "Insurance_Type": {"Meddicare":"Medicare", "Privatte":"Private"},
    "Hospital_Region": {"Southh":"South"}      
}

for col, mapping in typo_fix.items():
    df.loc[df[col].notna(), col] = df.loc[df[col].notna(), col].replace(mapping)

# After normalization, coerce any unknowns to NaN for consistent downstream handling
df.loc[~df["Hospital_Region"].isin(VALID_REGIONS), "Hospital_Region"] = pd.NA
df.loc[~df["Insurance_Type"].isin(VALID_INSURANCE), "Insurance_Type"] = pd.NA
df.loc[~df["Primary_Language"].isin(VALID_LANGS),   "Primary_Language"] = pd.NA


In [5]:
df["Admission_Date"] = pd.to_datetime(df["Admission_Date"], errors="coerce")

# If only a small % are NaT, fill with median date
date_median = df["Admission_Date"].median()
df["Admission_Date"] = df["Admission_Date"].fillna(date_median)

# Keep a clean date column normalized (no time component)
df["Admission_Date"] = df["Admission_Date"].dt.normalize()


In [6]:
# Categorical fills
for col in ["Insurance_Type", "Primary_Language", "Diagnosis_Code"]:
    df[col] = df[col].fillna("Unknown")

# Region: impute with mode (most frequent)
region_mode = df["Hospital_Region"].mode(dropna=True)
if len(region_mode):
    df["Hospital_Region"] = df["Hospital_Region"].fillna(region_mode.iloc[0])
else:
    df["Hospital_Region"] = df["Hospital_Region"].fillna("Unknown")

# Cost: median impute
df["Cost_of_Stay"] = df["Cost_of_Stay"].astype("float")
df["Cost_of_Stay"] = df["Cost_of_Stay"].fillna(df["Cost_of_Stay"].median())


In [7]:
df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
df["Length_of_Stay"] = pd.to_numeric(df["Length_of_Stay"], errors="coerce")

# Clip to plausible bounds
df["Age"] = df["Age"].clip(lower=0, upper=120)
df["Length_of_Stay"] = df["Length_of_Stay"].clip(lower=0.1, upper=60.0)

# If any became NaN due to coercion, impute with medians
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Length_of_Stay"] = df["Length_of_Stay"].fillna(df["Length_of_Stay"].median())


In [ ]:
q01, q99 = df["Cost_of_Stay"].quantile([0.01, 0.99])
df["Cost_of_Stay"] = df["Cost_of_Stay"].clip(lower=q01, upper=q99)
